In [197]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
#import geopandas as gpd
import re
import sklearn
import numpy as np
from sklearn.neighbors import NearestNeighbors
%matplotlib inline
plt.rcParams['figure.figsize'] = (12, 5);
sns.set_style('whitegrid')

# User Base Recommendation

## Assign weights to each feature

### city_id is supper critical, so we assign large weight. (customer_array[:,5]*=100.) Note that latitude_x and	longitude_x is also important but originally has large variance, so we keep them the same.)
### We want to amplify the customer's preference (customer_array[:,6]*=3.)
### grand_total and	item_count may not be so important, we dilute them (customer_array[:,-2:]/=3.)

### For each query we generate three results, two of them are based on the user's preference. The last one are on food categories that's less frequent

In [240]:
customer_array = np.load("customer_array.npy",allow_pickle=True)
customer_rare_array = np.load("customer_rare_array.npy",allow_pickle=True)
by_customer_rare = pd.read_csv('./by_customer_rare.csv')
by_customer = pd.read_csv('./by_customer.csv')

## fit KNN model

In [241]:
'''
samples = [[0, 0, 2], [1, 0, 0], [0, 0, 1]]
neigh = NearestNeighbors(2, 0.4)
neigh.fit(samples) 
neigh.kneighbors([[0, 0, 1.3]], 2, return_distance=False)
'''
neigh = NearestNeighbors(2, 0.4)
neigh.fit(customer_array) 

neigh_rare = NearestNeighbors(2, 0.4)
neigh_rare.fit(customer_rare_array) 

C:\Users\anban\anaconda3\lib\site-packages\sklearn\utils\validation.py:70: FutureWarning: Pass n_neighbors=2, radius=0.4 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "
C:\Users\anban\anaconda3\lib\site-packages\sklearn\utils\validation.py:70: FutureWarning: Pass n_neighbors=2, radius=0.4 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "


NearestNeighbors(n_neighbors=2, radius=0.4)

## Predict

In [242]:
neigh.kneighbors([[0, 0, 2, 0.0329, -78.6, 10000.0, 9.0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.7888888888888889,
       0.2222222222222222]], 2, return_distance=True)

(array([[9900.00347569, 9900.00366229]]), array([[34718,  6347]], dtype=int64))

In [243]:
neigh.kneighbors([[0, 0, 2, 0.0329, -78.6, 10000.0, 9.0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.7888888888888889,
       0.2222222222222222]], 2, return_distance=True)

(array([[9900.0036272 , 9900.00381381]]), array([[34718,  6347]], dtype=int64))

In [244]:
neigh.kneighbors([[0, 0, 2, 0.0329, -78.6, 10000.0, 9.0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.0,
       0.2222222222222222]], 2, return_distance=True)

(array([[9900.00384829, 9900.00423388]]), array([[34718, 34268]], dtype=int64))

## Give Recommendation

In [252]:
by_customer


,Unnamed: 0,VENDOR
customer_id,,
92PEE24,0,105 105 105 105
QS68UD8,1,294 43
MB7VY5F,2,83 83 83 83
KDJ951Y,3,90
BAL0RVT,4,83 83 83 83 84 84 216 216
...,...,...
0DSR94W,27440,84
3S6VG6R,27441,199
ND4PIJL,27442,907


In [253]:
by_customer = by_customer.set_index('customer_id')

KeyError: "None of ['customer_id'] are in the columns"

In [254]:
by_customer.loc["92PEE24"]

Unnamed: 0                  0
VENDOR        105 105 105 105
Name: 92PEE24, dtype: object

In [248]:
by_customer_rare = by_customer_rare.set_index('customer_id')

In [249]:
q = [[0, 0, 2, 0.0329, -78.6, 10000.0, 9.0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.0,
       0.2222222222222222]]

def query(user_vecs, neigh, table, pool, k_value=3, return_num=1):
    dist, knn = neigh.kneighbors(user_vecs, k_value, return_distance=True)
    recommendation = []
    recommendation_weight = []
    #print(dist,knn)
    for k,i in enumerate(list(knn)):
        
        #print(pool.iloc[i])
        user_id = table.iloc[i]['customer_id'].values
        recommended_restaurant = []
        for id in user_id:
            recommended_restaurant.append(pool.loc[id]['VENDOR'].split(' '))
        confidence = {}
        for j in range(len(recommended_restaurant)):
            for restaurant in recommended_restaurant[j]:
                if restaurant not in confidence:
                    confidence[restaurant]=0.
                confidence[restaurant]+=1./dist[0][j]
        p = np.asarray(list(confidence.values()))
        p/=np.sum(p)
        keys = list(confidence.keys())
        recommendation.append(np.random.choice(len(confidence.keys()), return_num, p=p,replace=False))
        ret = [keys[recommendation[-1][n]]for n in range(return_num)]
        #print("generate recommendation for %d'th input..."%k)
        #print("--Recommendation is restaurant: "+str(ret))
        return ret
        

def generate_recommendation(user_vecs):
    l1 = query(user_vecs, neigh, customer, by_customer, k_value=3, return_num=2)
    l2 = query(user_vecs, neigh_rare, customer_rare, by_customer_rare, k_value=3, return_num=1)
    l1.extend(l2)
    return l1

generate_recommendation(q)

['304', '43', '243']

# Generate Recommendation for Group 

In [250]:
def generate_recommendation_for_group(user_vecs):
    hybrid_preference = np.sum(np.asarray(user_vecs)[:,6:-2],0).tolist()
    vote = []
    vote_rare = []
    count_freq = {}
    for user_vec in user_vecs:
        user_vec[6:-2]=hybrid_preference
        recommendation = generate_recommendation([user_vec])
        vote.extend(recommendation[:2])
        vote_rare.extend(recommendation[2:])
    for k in vote:
        if k not in count_freq:
            count_freq[k]=0
        count_freq[k]+=1
    count_freq = sorted(count_freq.items(),key=lambda x:x[1], reverse=True)
    ret =[i[0] for i in count_freq[:2]]
    count_freq = {}
    for k in vote_rare:
        if k not in count_freq:
            count_freq[k]=0
        count_freq[k]+=1
    count_freq = sorted(count_freq.items(),key=lambda x:x[1], reverse=True)
    ret.append(count_freq[0][0])
    #print("generate recommendation for group...")
    #print("--Recommendation is restaurant: "+str(ret))
    return ret

In [251]:
generate_recommendation_for_group(customer_array[:10].tolist())

['216', '221', '243']

# Item Base Recommendation

## Assign weights to each feature

### city_id is supper critical, so we assign large weight. (customer_array[:,5]*=100.) Note that latitude_x and	longitude_x is also important but originally has large variance, so we keep them the same.)
### We want to amplify the customer's preference (customer_array[:,6]*=3.)
### grand_total and	item_count may not be so important, we dilute them (customer_array[:,-2:]/=3.)